In [1]:
from gensim.models.doc2vec import *

In [2]:
model = Doc2Vec.load('../store/doc2vec_model_800_pico_19k.mod')

In [3]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
cohen_vec = cPickle.load(open('../../preprocess/cohendata_dedup_19995.p'))

Using TensorFlow backend.


[]


In [4]:
X = cohen_vec
X = map(lambda s : [cohen_vec.idx2word[t] for t in s if t > 0], X)

In [5]:
embedds = []
for sent in X :
    embedds.append(model.infer_vector(sent))

In [6]:
import numpy as np
embedds = np.array(embedds)
embedds.shape

(1767, 800)

In [7]:
import pandas as pd
df = pd.read_csv('../data/files/test_cohen_dedup.csv')

nb_studies = len(df)
H = np.zeros((nb_studies, nb_studies))

cdnos = list(set(df.cdno))
for i in range(nb_studies) :
    H[i, df[df['cdno'] == df['cdno'][i]].index] = 1
    
H[np.arange(nb_studies), np.arange(nb_studies)] = 0

In [8]:
from sklearn.preprocessing import normalize
embedds_n = normalize(embedds, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [10]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.75958038059
ACEInhibitors_processed              0.736109
ADHD_processed                       0.823953
Antihistamines_processed             0.729723
AtypicalAntipsychotics_processed     0.851343
BetaBlockers_processed               0.647976
CalciumChannelBlockers_processed     0.596768
Estrogens_processed                  0.848794
NSAIDS_processed                     0.770599
Opiods_processed                     0.748220
OralHypoglycemics_processed          0.799494
ProtonPumpInhibitors_processed       0.792898
SkeletalMuscleRelaxants_processed    0.672570
Statins_processed                    0.756744
Triptans_processed                   0.822503
dtype: float64
